# OpenStack Example LRZ


Installation Packages:

    conda install -c conda-forge boto3  pykafka pyspark  dask distributed python-
    confluent-kafka pexpect redis-py 
    conda install paramiko distributed
    conda install -c conda-forge python-openstackclient
    
    
https://iujetstream.atlassian.net/wiki/spaces/JWT/pages/730234884/Installing+the+Openstack+clients+on+Linux

Rename and complete `openstack_description.json.template` to `openstack_description.json.`

In [ ]:
import os, sys, json, time
from cinderclient import client
from keystoneauth1 import loading
from keystoneauth1 import session
import getpass
import openstack
from novaclient.client import Client

In [ ]:
os_password = getpass.getpass()

In [ ]:
os_description = json.load(open("openstack_description_lrz.json", "r"))
loader = loading.get_plugin_loader('password')
auth = loader.load_from_options(auth_url=os_description["os_auth_url"],
                                username=os_description["os_username"],
                                password=os_password,
                                default_domain_name=os_description["os_user_domain"],
                                project_id=os_description["os_project_id"])
sess = session.Session(auth=auth)


In [ ]:
conn = openstack.connection.Connection(session=sess)

In [ ]:
conn.list_servers()[:2]

In [ ]:
#for i in conn.compute.images(): print(i)


Create VM Instance

In [ ]:
server = conn.create_server(name=os_description["os_name"],
                   image=os_description["os_image_id"], 
                   flavor=os_description["os_instance_type"], 
                   key_name=os_description["os_ssh_keyname"],
                   security_groups=[os_description["os_security_group"]],
                   auto_ip=True, 
                   ips=None,
                   ip_pool=None, 
                   root_volume=None, 
                   terminate_volume=True, 
                   wait=False, 
                   timeout=180, 
                   reuse_ips=True, 
                   network= os_description["os_network"],
                   boot_from_volume=True,
                   volume_size='20', 
                   boot_volume=None, 
                   volumes=None,
                   nat_destination=None, 
                   group=None)


Wait for server

In [ ]:
s = "UNKNOWN"
while s != "ACTIVE":
    s=conn.update_server(name_or_id=server["id"])["status"]
    time.sleep(2)
    print("Server: {} Status: {}".format(server["id"], s))

Allocate IP

In [ ]:
ip=conn.create_floating_ip(network=os_description["os_network_floating_ip"],
                           server=server)
print(ip)

List Servers

In [ ]:
servers=conn.list_servers()
servers

Stop servers and release resources

In [ ]:
if False:
    print("Delete server {}".format(server["id"]))
    conn.delete_server(name_or_id=server["id"], delete_ips=True)
    conn.delete_floating_ip(ip)
    conn.close()
else:
    print("Server still running")